In [205]:
import numpy as np
import pandas as pd

## Frequent Item sets

In [331]:
transaction=pd.read_clipboard(header=None)
transaction

,0,1
0,T1,"I1,I2,I3"
1,T2,"I2,I3,I4"
2,T3,"I4,I5"
3,T4,"I1,I2,I4"
4,T5,"I1,I2,I3,I5"
5,T6,"I1,I2,I3,I4"


In [352]:
transactions=[x.split(',') for x in transaction[1]]
transactions=[[chr(int(i[1])+64) for i in x] for x in transactions]
transactions

[['A', 'B', 'C'],
 ['B', 'C', 'D'],
 ['D', 'E'],
 ['A', 'B', 'D'],
 ['A', 'B', 'C', 'E'],
 ['A', 'B', 'C', 'D']]

In [362]:
support=3

In [363]:
def recurse(lst,count,curr,k,freq):
    n=len(lst)
    if len(curr)==k:
        if tuple(curr) in freq:
            freq[tuple(curr)]+=1
    else:
        l=len(curr)
        if l==0 or tuple(curr) in fps[l]:
            for i in range(count+1,n):
                curr=[j for j in curr[:l]]
                curr.append(lst[i])
                recurse(lst,i,curr,k,freq)

### Step 1 : Finding frequent pairs

In [364]:
candidates=dict()
    
for i in transactions:
    for j in i:
        if tuple(j,) not in candidates:
            candidates[tuple(j,)]=1
        else:
            candidates[tuple(j,)]+=1


freq=dict()
for i in candidates:
    if candidates[i]>=support:
        freq[tuple(i)]=candidates[i]

cps=dict()
fps=dict()
cps[1]=candidates
fps[1]=freq
print("Candidates :")
print(cps)
print("Frequent candidates :")
print(fps)

Candidates :
{1: {('A',): 4, ('B',): 5, ('C',): 4, ('D',): 4, ('E',): 2}}
Frequent candidates :
{1: {('A',): 4, ('B',): 5, ('C',): 4, ('D',): 4}}


### Step 2

In [365]:
k=2
new=1
while(new!=0):
    new=0
    cand=dict()
    vals=list(fps[k-1].keys())
    vals.sort()
    n=len(vals)
    for i in range(n):
        s=list(vals[i])
        for j in range(i+1,n):
            s1=list(vals[j])
            s.extend(s1)
            st=set(s)
            if len(st)==k:
                tp=list(st)
                tp.sort()
                if tuple(tp) not in cand:
                    cand[tuple(tp)]=0
            s=list(vals[i])
    curr=[]
    for j in transactions:
        recurse(j,-1,curr,k,cand)
    print("Candidate Pairs :")
    print(cand)
    if cand=={}:
        new=0
    else:
        z=list(cand.keys())[0]
        print("Frequent Pairs :")
        freq=dict()
        for i in cand:
            if cand[i]>=support:
                print(i,cand[i])
                new=1
                freq[i]=cand[i]
        z=len(list(cand.keys())[0])

        print(z)
        print(new)
        fps[z]=freq
        k+=1

Candidate Pairs :
{('A', 'B'): 4, ('A', 'C'): 3, ('A', 'D'): 2, ('B', 'C'): 4, ('B', 'D'): 3, ('C', 'D'): 2}
Frequent Pairs :
('A', 'B') 4
('A', 'C') 3
('B', 'C') 4
('B', 'D') 3
2
1
Candidate Pairs :
{('A', 'B', 'C'): 3, ('A', 'B', 'D'): 2, ('B', 'C', 'D'): 2}
Frequent Pairs :
('A', 'B', 'C') 3
3
1
Candidate Pairs :
{}


### Frequent Pairs

In [366]:
fps

{1: {('A',): 4, ('B',): 5, ('C',): 4, ('D',): 4},
 2: {('A', 'B'): 4, ('A', 'C'): 3, ('B', 'C'): 4, ('B', 'D'): 3},
 3: {('A', 'B', 'C'): 3}}

### Association rules

In [367]:
total=len(fps)
print(total)

3


In [368]:
import itertools
generate = [0,1]
trans_len=len(transactions)
X=[]
Y=[]
supp=[]
conf=[]
lift=[]



for i in range(2,total+1):
    itemsets=fps[i]
    for s in itemsets:
        l=len(s)
        combs=[p for p in itertools.product(generate, repeat=l)]
        for j in range(1,len(combs)-1):
            rule=combs[j]
            x=[]
            y=[]
            for k in range(l):
                if rule[k]==0:
                    x.append(s[k])
                else:
                    y.append(s[k])
                l1=len(x)
                l2=len(y)
            num=itemsets[s]
            den=fps[l1][tuple(x)]
            lfcount=fps[l2][tuple(y)]
            supp.append(num/trans_len)
            conf.append(num/den)
            lift.append((num/den)*trans_len/lfcount)
            X.append(tuple(x))
            Y.append(tuple(y))
print("Generated association rules")

Generated association rules


In [369]:
n_rules=len(X) 
lsts=[["X","Y","Support","Confidence","Lift"]]
for i in range(n_rules):
    x="("+str(X[i][0])
    for it in X[i][1:]:
        x+=","+str(it)
    x+=")"
    y="("+str(Y[i][0])
    for it in Y[i][1:]:
        y+=","+str(it)
    y+=")"  
    s=""+str(x)+" "+str(y)+" "+'%.2f'%supp[i]+" "+'%.2f'%conf[i]+" "+'%.2f'%lift[i]
    lsts.append(s.split())
sizes = [max(len(item) for item in l) + 2 for l in zip(*lsts)]

for lst in lsts:
    print("".join(word.ljust(sizes[i]) for i, word in enumerate(lst)))

X      Y      Support  Confidence  Lift  
(A)    (B)    0.67     1.00        1.20  
(B)    (A)    0.67     0.80        1.20  
(A)    (C)    0.50     0.75        1.12  
(C)    (A)    0.50     0.75        1.12  
(B)    (C)    0.67     0.80        1.20  
(C)    (B)    0.67     1.00        1.20  
(B)    (D)    0.50     0.60        0.90  
(D)    (B)    0.50     0.75        0.90  
(A,B)  (C)    0.50     0.75        1.12  
(A,C)  (B)    0.50     1.00        1.20  
(A)    (B,C)  0.50     0.75        1.12  
(B,C)  (A)    0.50     0.75        1.12  
(B)    (A,C)  0.50     0.60        1.20  
(C)    (A,B)  0.50     0.75        1.12  
